# Create Time-Series ML Model

## Load libraries

First, we will load the libraries.

In [61]:
import pandas as pd
import pprint
import logging
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.metrics import mean_absolute_error, mean_squared_error, r2_score
from xgboost import XGBRegressor

pp = pprint.PrettyPrinter(indent=2)

logging.basicConfig(
    level=logging.INFO,
    format="%(asctime)s - %(levelname)s - %(message)s",
    datefmt="%Y-%m-%d %H:%M:%S",
)

## Prepare data

Then we load, and prepare the data.

In [62]:
def sort_by_year_repetition(df):
    # Extract election id components from the index
    df['year'] = df.index.map(lambda x: int(x.split('_')[0][1:5]))  # Assumes year is four digits long
    df['repetitionid'] = df.index.map(lambda x: int(x.split('_')[0][5:]))  # Assumes repetitionid immediately follows year

    # Sort the DataFrame by year and then by repetitionid
    df_sorted = df.sort_values(by=['year', 'repetitionid'])

    # Drop the temporary columns used for sorting
    df_sorted = df_sorted.drop(columns=['year', 'repetitionid'])

    return df_sorted

In [63]:
def add_shifted_columns_grouped(df):
    # Extract 'mundissec' from the index
    df['mundissec'] = df.index.map(lambda x: x.split('_')[1])

    # Identify unique party codes by splitting each column name
    party_codes = set(col.split('_')[-1] for col in df.columns if ('_' in col) and (col.split('_')[-1].isupper()))

    # Iterate over each party code to create shifted columns group-wise
    for party_code in party_codes:
        # Identify columns for the current party code
        party_columns = [col for col in df.columns if col.endswith(party_code)]
        for col in party_columns:
            # Create a new shifted column name
            shifted_col_name = f"{col}_shifted"
            # Group by 'mundissec' and shift within each group
            df[shifted_col_name] = df.groupby('mundissec')[col].shift(-1)  # Shift within each group

    # Drop the temporary 'mundissec' column after shifting
    df.drop(columns='mundissec', inplace=True)

    return df

In [64]:
df_timeseries = pd.read_pickle("../../data/output/timeseries_2010_2024_6_1_True_True_True_True.pkl")
df_sorted = sort_by_year_repetition(df_timeseries)
df_timeseries = add_shifted_columns_grouped(df_sorted)
df_timeseries.head()

,cens_electoral_percentage_ALTRES,cens_electoral_percentage_CS,cens_electoral_percentage_CUP,cens_electoral_percentage_ERC,cens_electoral_percentage_JXCATJUNTS,cens_electoral_percentage_PODEMOS,cens_electoral_percentage_PP,cens_electoral_percentage_PSC,cens_electoral_percentage_VOX,election_type,...,2020_ist,cens_electoral_percentage_PP_shifted,cens_electoral_percentage_CUP_shifted,cens_electoral_percentage_PODEMOS_shifted,cens_electoral_percentage_PSC_shifted,cens_electoral_percentage_ALTRES_shifted,cens_electoral_percentage_CS_shifted,cens_electoral_percentage_VOX_shifted,cens_electoral_percentage_JXCATJUNTS_shifted,cens_electoral_percentage_ERC_shifted
electionid_mundissec,,,,,,,,,,,,,,,,,,,,,
A20101_08001801001,0.333309,2.555366,0.0,3.577513,17.802385,5.451448,7.240204,14.821124,0.0,0,...,100.6,13.613014,0.0,5.821918,23.458904,0.880626,0.0,0.0,13.698630,4.023973
A20101_08001801002,0.253207,1.762452,0.0,1.992337,15.402299,3.448276,9.808429,17.164751,0.0,0,...,93.5,16.156863,0.0,5.490196,25.882353,0.941176,0.0,0.0,9.490196,2.352941
A20101_08001801003,0.295716,4.595588,0.0,3.676471,22.610294,5.330882,8.088235,11.213235,0.0,0,...,117.1,16.546763,0.0,5.575540,22.122302,0.796506,0.0,0.0,20.683453,2.158273
A20101_08001801004,0.235790,2.793755,0.0,4.765818,21.446179,5.587510,6.080526,13.064914,0.0,0,...,113.7,16.494845,0.0,6.344171,20.856463,0.657075,0.0,0.0,17.287867,3.647898
A20101_08001801005,0.276506,1.900585,0.0,2.558480,14.473684,6.725146,8.552632,14.254386,0.0,0,...,99.3,16.188374,0.0,7.211185,23.031641,0.998633,0.0,0.0,9.050773,2.796174


In [65]:
df_timeseries.tail(5)

,cens_electoral_percentage_ALTRES,cens_electoral_percentage_CS,cens_electoral_percentage_CUP,cens_electoral_percentage_ERC,cens_electoral_percentage_JXCATJUNTS,cens_electoral_percentage_PODEMOS,cens_electoral_percentage_PP,cens_electoral_percentage_PSC,cens_electoral_percentage_VOX,election_type,...,2020_ist,cens_electoral_percentage_PP_shifted,cens_electoral_percentage_CUP_shifted,cens_electoral_percentage_PODEMOS_shifted,cens_electoral_percentage_PSC_shifted,cens_electoral_percentage_ALTRES_shifted,cens_electoral_percentage_CS_shifted,cens_electoral_percentage_VOX_shifted,cens_electoral_percentage_JXCATJUNTS_shifted,cens_electoral_percentage_ERC_shifted
electionid_mundissec,,,,,,,,,,,,,,,,,,,,,
A20211_43906001002,0.285423,2.038736,2.548420,11.009174,20.897044,1.630989,1.732926,8.358818,2.854230,0,...,86.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A20211_43907601001,0.161017,3.389831,1.610169,8.389831,5.423729,3.644068,2.627119,15.338983,5.084746,0,...,105.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A20211_43907601002,0.123023,3.690685,0.615114,4.393673,0.790861,2.372583,2.372583,20.562390,8.875220,0,...,97.5,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A20211_43907601003,0.118534,3.232759,2.262931,7.974138,6.573276,2.586207,2.586207,17.133621,5.711207,0,...,99.9,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
A20211_43907601004,0.127226,2.544529,2.120441,5.937235,3.307888,2.374894,1.357082,15.945717,7.294317,0,...,96.2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [66]:
def split_data(df):
    # Identify unique elections from the index
    elections = df.index.map(lambda x: x.split('_')[0]).unique()

    # Latest and penultimate election identifiers
    last_election = elections[-1]
    penultimate_election = elections[-2] if len(elections) > 1 else None

    # Split the DataFrame based on the election identifiers
    new_data = df.loc[df.index.map(lambda x: x.split('_')[0]) == last_election]
    test_data = df.loc[df.index.map(lambda x: x.split('_')[0]) == penultimate_election] if penultimate_election else pd.DataFrame()
    train_data = df.loc[~df.index.map(lambda x: x.split('_')[0]).isin([last_election, penultimate_election])]

    # Columns that are not shifted
    non_shifted_columns = [col for col in df.columns if not col.endswith('_shifted')]
    # Columns that are shifted
    shifted_columns = [col for col in df.columns if col.endswith('_shifted')]

    # Creating new_data, X_test, y_test, X_train, y_train
    new_data = new_data[non_shifted_columns]
    X_test = test_data[non_shifted_columns]
    y_test = test_data[shifted_columns]
    X_train = train_data[non_shifted_columns]
    y_train = train_data[shifted_columns]

    return X_train, y_train, X_test, y_test, new_data

# Test the function
X_train, y_train, X_test, y_test, new_data = split_data(df_timeseries)

# Printing shapes to verify the splits
print(f'X_train shape: {X_train.shape}, y_train shape: {y_train.shape}')
print(f'X_test shape: {X_test.shape}, y_test shape: {y_test.shape}')
print(f'New data shape: {new_data.shape}')

X_train shape: (66079, 17), y_train shape: (66079, 9)
X_test shape: (5083, 17), y_test shape: (5083, 9)
New data shape: (5083, 17)


In [67]:
X_train = X_train.astype(float)
y_train = y_train.astype(float)
X_test = X_test.astype(float)
y_test = y_test.astype(float)
new_data = new_data.astype(float)

In [81]:
# Set up the parameters for XGBoost
params_multioutput = {
    "n_estimators": 400,  # Number of boosting rounds
    "max_depth": 3,  # Typically 3-10. Higher values can lead to overfitting.
    "eta": 0.01,  # Learning rate, typically between 0.01 and 0.2
    "objective": "reg:squarederror",  # Regression with squared loss
    "eval_metric": "rmse",  # Root Mean Square Error for evaluation
    "tree_method": "hist",  # Fast histogram optimized approximate greedy algorithm
    "multi_strategy": "multi_output_tree",
    "early_stopping_rounds": 5,
    "reg_alpha": 1,  # L1 regularization term on weights. Increasing this value will make model more conservative. 
    "reg_lambda": 1,  # L2 regularization term on weights. Increasing this value will make model more conservative.
}
# Define the model_multioutput
model_multioutput = XGBRegressor(**params_multioutput)

# Train the model_multioutput
eval_set = [(X_test, y_test)]
model_multioutput.fit(X_train, y_train, eval_set=eval_set, verbose=True)

[0]	validation_0-rmse:5.73040
[1]	validation_0-rmse:5.68170
[2]	validation_0-rmse:5.63166
[3]	validation_0-rmse:5.58454
[4]	validation_0-rmse:5.53621
[5]	validation_0-rmse:5.49023
[6]	validation_0-rmse:5.44495
[7]	validation_0-rmse:5.39985
[8]	validation_0-rmse:5.35548
[9]	validation_0-rmse:5.31171
[10]	validation_0-rmse:5.26986
[11]	validation_0-rmse:5.22706
[12]	validation_0-rmse:5.18593
[13]	validation_0-rmse:5.14457
[14]	validation_0-rmse:5.10441
[15]	validation_0-rmse:5.06585
[16]	validation_0-rmse:5.02647
[17]	validation_0-rmse:4.98999
[18]	validation_0-rmse:4.95145
[19]	validation_0-rmse:4.91182
[20]	validation_0-rmse:4.87460
[21]	validation_0-rmse:4.83397
[22]	validation_0-rmse:4.79793
[23]	validation_0-rmse:4.76324
[24]	validation_0-rmse:4.72903
[25]	validation_0-rmse:4.68855
[26]	validation_0-rmse:4.65632
[27]	validation_0-rmse:4.61643
[28]	validation_0-rmse:4.58546
[29]	validation_0-rmse:4.55319
[30]	validation_0-rmse:4.51944
[31]	validation_0-rmse:4.49253
[32]	validation_0-

XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=5,
             enable_categorical=False, eta=0.01, eval_metric='rmse',
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=3,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy='multi_output_tree',
             n_estimators=400, n_jobs=None, num_parallel_tree=None, ...)

In [82]:
predictions = model_multioutput.predict(X_test, iteration_range=(0, model_multioutput.best_iteration + 1))
if not isinstance(predictions, pd.DataFrame):
    predictions = pd.DataFrame(predictions, index=y_test.index, columns=y_test.columns)

# Initialize a dictionary to store metrics for each column
metrics = {}

# Loop through each column in y_test to calculate metrics
for column in y_test.columns:
    mae = mean_absolute_error(y_test[column], predictions[column])
    mse = mean_squared_error(y_test[column], predictions[column])
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test[column], predictions[column])

    # Store metrics in the dictionary
    metrics[column] = {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R^2': r2}

# Print or return the metrics
for col, values in metrics.items():
    print(f"Metrics for {col}:")
    print(f"  MAE: {values['MAE']}")
    print(f"  MSE: {values['MSE']}")
    print(f"  RMSE: {values['RMSE']}")
    print(f"  R^2: {values['R^2']}\n")

Metrics for cens_electoral_percentage_PP_shifted:
  MAE: 2.740878429331179
  MSE: 8.992723848152648
  RMSE: 2.998787062822675
  R^2: -2.5293457458711655

Metrics for cens_electoral_percentage_CUP_shifted:
  MAE: 1.686876891675421
  MSE: 5.508967906967431
  RMSE: 2.3471190653580893
  R^2: 0.22212110422133524

Metrics for cens_electoral_percentage_PODEMOS_shifted:
  MAE: 2.8915911292747882
  MSE: 11.040950949640708
  RMSE: 3.3227926431904695
  R^2: -2.434685958447783

Metrics for cens_electoral_percentage_PSC_shifted:
  MAE: 2.116763736804178
  MSE: 7.771283916336629
  RMSE: 2.787702264650339
  R^2: 0.6578591282544655

Metrics for cens_electoral_percentage_ALTRES_shifted:
  MAE: 2.002277800411714
  MSE: 4.080943680836235
  RMSE: 2.020134569981969
  R^2: -71.72601329116189

Metrics for cens_electoral_percentage_CS_shifted:
  MAE: 2.8563078641990427
  MSE: 9.458747237894581
  RMSE: 3.0755076390564504
  R^2: -3.2240228443284815

Metrics for cens_electoral_percentage_VOX_shifted:
  MAE: 1.94

This results are promising, as we got good $R^2$ values for 4 parties. These parties are:
- JxCat (1031): $R^2 = 0.820$
- PSC (6): $R^2 = 0.569$
- CUP (1003): $R^2 = 0.445$
- ERC (10): $R^2 = 0.471$

We've added the election type for each election, as this is an important feature. The results are similar, but slighly worse:

- JxCat (1031): $R^2 = 0.819$
- PSC (6): $R^2 = 0.241$
- CUP (1003): $R^2 = 0.329$
- ERC (10): $R^2 = 0.681$

We've also added the proportion of born abroad and the proportion of different groups of age. The results are similar, but they have slightly improved for PSC, and ERC, and slightly worsened for JxCat and CUP. The results are as follows:

- JxCat (1031): $R^2 = 0.800$
- PSC (6): $R^2 = 0.479$
- CUP (1003): $R^2 = 0.299$
- ERC (10): $R^2 = 0.443$

We've also added the mean income data by census section. The results are again similar, but slightly worse, as the $R^2$ values are very low. That is something strange because the mean income is a very important feature in the prediction of the election results. The results are as follows:

- JxCat (1031): $R^2 = 0.805$
- PSC (6): $R^2 = 0.463$
- CUP (1003): $R^2 = 0.261$
- ERC (10): $R^2 = 0.461$

In [70]:
# Set up the parameters for XGBoost
params = {
    'max_depth': 6,  # Typically 3-10. Higher values can lead to overfitting.
    'eta': 0.1,  # Learning rate, typically between 0.01 and 0.2
    'objective': 'reg:squarederror',  # Regression with squared loss
    'eval_metric': 'rmse',  # Root Mean Square Error for evaluation
    "early_stopping_rounds": 10, 
}
num_boost_round = 200  # Number of boosting rounds

# Define the model
model = XGBRegressor(**params)

# Train the model
eval_set = [(X_test, y_test)]
model.fit(X_train, y_train, eval_set=eval_set, verbose=True)

[0]	validation_0-rmse:5.19839
[1]	validation_0-rmse:4.70118
[2]	validation_0-rmse:4.25681
[3]	validation_0-rmse:3.89885
[4]	validation_0-rmse:3.62585
[5]	validation_0-rmse:3.39779
[6]	validation_0-rmse:3.21825
[7]	validation_0-rmse:3.08521
[8]	validation_0-rmse:3.00417
[9]	validation_0-rmse:2.94421
[10]	validation_0-rmse:2.91757
[11]	validation_0-rmse:2.92063
[12]	validation_0-rmse:2.93477
[13]	validation_0-rmse:2.96775
[14]	validation_0-rmse:2.98826
[15]	validation_0-rmse:3.02384
[16]	validation_0-rmse:3.09098
[17]	validation_0-rmse:3.13282
[18]	validation_0-rmse:3.18949
[19]	validation_0-rmse:3.23809
[20]	validation_0-rmse:3.27586


XGBRegressor(base_score=None, booster=None, callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device=None, early_stopping_rounds=10,
             enable_categorical=False, eta=0.1, eval_metric='rmse',
             feature_types=None, gamma=None, grow_policy=None,
             importance_type=None, interaction_constraints=None,
             learning_rate=None, max_bin=None, max_cat_threshold=None,
             max_cat_to_onehot=None, max_delta_step=None, max_depth=6,
             max_leaves=None, min_child_weight=None, missing=nan,
             monotone_constraints=None, multi_strategy=None, n_estimators=None,
             n_jobs=None, num_parallel_tree=None, ...)

In [71]:
# Assuming 'predictions' is a DataFrame or array with the same structure as 'y_test'
predictions = model.predict(X_test)
if not isinstance(predictions, pd.DataFrame):
    predictions = pd.DataFrame(predictions, index=y_test.index, columns=y_test.columns)

# Initialize a dictionary to store metrics for each column
metrics = {}

# Loop through each column in y_test to calculate metrics
for column in y_test.columns:
    mae = mean_absolute_error(y_test[column], predictions[column])
    mse = mean_squared_error(y_test[column], predictions[column])
    rmse = np.sqrt(mse)
    r2 = r2_score(y_test[column], predictions[column])

    # Store metrics in the dictionary
    metrics[column] = {'MAE': mae, 'MSE': mse, 'RMSE': rmse, 'R^2': r2}

# Print or return the metrics
for col, values in metrics.items():
    print(f"Metrics for {col}:")
    print(f"  MAE: {values['MAE']}")
    print(f"  MSE: {values['MSE']}")
    print(f"  RMSE: {values['RMSE']}")
    print(f"  R^2: {values['R^2']}\n")

Metrics for cens_electoral_percentage_PP_shifted:
  MAE: 2.4422469759432865
  MSE: 7.151817602815382
  RMSE: 2.674288242283427
  R^2: -1.8068511229696247

Metrics for cens_electoral_percentage_CUP_shifted:
  MAE: 2.0810538200702915
  MSE: 5.852199182209505
  RMSE: 2.4191319067404127
  R^2: 0.17365606142369427

Metrics for cens_electoral_percentage_PODEMOS_shifted:
  MAE: 2.223166762076298
  MSE: 6.374353065162779
  RMSE: 2.524748119152241
  R^2: -0.9829723967586008

Metrics for cens_electoral_percentage_PSC_shifted:
  MAE: 2.80524113432418
  MSE: 11.85505277550355
  RMSE: 3.4431167240602734
  R^2: 0.4780658981364121

Metrics for cens_electoral_percentage_ALTRES_shifted:
  MAE: 3.6662114850913174
  MSE: 15.367742827548645
  RMSE: 3.9201712752823243
  R^2: -272.8667221456101

Metrics for cens_electoral_percentage_CS_shifted:
  MAE: 1.7263599948459616
  MSE: 4.281652930449668
  RMSE: 2.069215535039709
  R^2: -0.9120713700064038

Metrics for cens_electoral_percentage_VOX_shifted:
  MAE: 1.

This results are promising, as we got good $R^2$ values for 4 parties. These parties are:
- JxCat (1031): $R^2 = 0.709$
- PSC (6): $R^2 = 0.726$
- CUP (1003): $R^2 = 0.419$
- ERC (10): $R^2 = 0.444$

This means that in some cases the past percentage of votes explains a lot of the future percentage of votes. This is a good sign, as we can use this information to predict the future. But we need to be careful, as this is not always the case.

This parties have in common that they are important, and have consistent data for the past years. Some parties have surprising behaviours that are not explained by the past data.

We've added the election type for each election, as this is an important feature. The results are similar, but slighly worse:

- JxCat (1031): $R^2 = 0.738$
- PSC (6): $R^2 = 0.679$
- CUP (1003): $R^2 = 0.402$
- ERC (10): $R^2 = 0.291$

## Predictions with new data

In [89]:
new_data.head()

,cens_electoral_percentage_ALTRES,cens_electoral_percentage_CS,cens_electoral_percentage_CUP,cens_electoral_percentage_ERC,cens_electoral_percentage_JXCATJUNTS,cens_electoral_percentage_PODEMOS,cens_electoral_percentage_PP,cens_electoral_percentage_PSC,cens_electoral_percentage_VOX,election_type,2021_p_born_abroad,2021_p_adult,2021_p_child,2021_p_senior,2021_p_young,2021_mean_income,2020_ist
electionid_mundissec,,,,,,,,,,,,,,,,,
A20211_08001801001,0.174978,4.549431,2.537183,11.723535,4.286964,5.336833,1.924759,16.885389,4.636920,0.0,0.147929,0.420118,0.134780,0.230112,0.214990,37743.0,100.6
A20211_08001801002,0.120048,3.666921,1.375095,9.472880,4.125286,4.583652,2.215432,18.792972,4.812834,0.0,0.149270,0.426717,0.166577,0.198486,0.208221,36847.0,93.5
A20211_08001801003,0.325733,5.700326,2.117264,13.355049,6.351792,5.374593,2.117264,18.403909,5.863192,0.0,0.064403,0.487119,0.183841,0.139344,0.189696,56376.0,117.1
A20211_08001801004,0.170906,3.870514,2.181562,12.596763,5.418719,5.277973,2.251935,17.663617,6.403941,0.0,0.077829,0.491551,0.197645,0.131592,0.179211,43789.0,113.7
A20211_08001801005,0.205698,4.607631,1.079914,7.847372,2.735781,3.167747,2.951764,19.006479,4.679626,0.0,0.118187,0.453859,0.145170,0.194819,0.206152,37459.0,99.3


In [83]:
# Make predictions
new_predictions = model.predict(new_data)
print(f"Predictions for New Data: {new_predictions}")

Predictions for New Data: [[ 4.490962   2.102921   5.616729  ...  2.8204975  3.958236   7.5763526]
 [ 4.715793   2.102921   5.478407  ...  2.9793525  4.2687116  2.0513415]
 [ 4.181255   2.102921   5.7105813 ...  2.7684157  4.491551  12.076935 ]
 ...
 [ 5.0130215  2.102921   5.932082  ...  3.1185665  4.7701716  3.7390099]
 [ 4.773823   2.102921   4.8065834 ...  3.1131074  6.782664   2.187056 ]
 [ 4.046435   2.102921   5.932082  ...  3.1131074  6.522464   3.3922076]]


In [84]:
# for each column in the predictions, calculate the mean
mean_predictions = new_predictions.mean(axis=0)
mean_predictions

array([4.4402575, 2.102891 , 5.6360598, 8.0361395, 2.183942 , 3.8547645,
       2.7195566, 9.197541 , 9.714045 ], dtype=float32)

In [85]:
mean_predictions = mean_predictions.reshape(1, -1)  # Reshape the array
mean_predictions_df = pd.DataFrame(mean_predictions, columns=y_test.columns)

In [86]:
mean_predictions_df

,cens_electoral_percentage_PP_shifted,cens_electoral_percentage_CUP_shifted,cens_electoral_percentage_PODEMOS_shifted,cens_electoral_percentage_PSC_shifted,cens_electoral_percentage_ALTRES_shifted,cens_electoral_percentage_CS_shifted,cens_electoral_percentage_VOX_shifted,cens_electoral_percentage_JXCATJUNTS_shifted,cens_electoral_percentage_ERC_shifted
0,4.440258,2.102891,5.63606,8.036139,2.183942,3.854764,2.719557,9.197541,9.714045


In [87]:
total_census = 5754840
total_votes = (mean_predictions_df/100) * total_census
total_votes

,cens_electoral_percentage_PP_shifted,cens_electoral_percentage_CUP_shifted,cens_electoral_percentage_PODEMOS_shifted,cens_electoral_percentage_PSC_shifted,cens_electoral_percentage_ALTRES_shifted,cens_electoral_percentage_CS_shifted,cens_electoral_percentage_VOX_shifted,cens_electoral_percentage_JXCATJUNTS_shifted,cens_electoral_percentage_ERC_shifted
0,255529.726125,121018.011459,324346.227555,462466.976584,125682.374943,221835.53657,156506.127601,529303.787262,559027.726015


## PyCaret

In [108]:
train_data = X_train.copy()
train_data[y_train.columns] = y_train
test_data = X_test.copy()
test_data[y_test.columns] = y_test

# Combine features and target columns for the setup
all_columns = X_train.columns.tolist() + y_train.columns.tolist()

In [109]:
from pycaret.regression import *

for col in y_train.columns:
    print(col)
    train_data = X_train.copy()
    train_data[col] = y_train[col]
    test_data = X_test.copy()
    test_data[col] = y_test[col]

    # Initialize the setup
    reg_setup = setup(
        data=train_data,
        test_data=test_data,
        target=col,
        session_id=123,
        normalize=True,
        transformation=True,
        transform_target=True,
        verbose=False,
    )
    best_model = compare_models()
    print(f"The best model for {col} is: {best_model}")

cens_electoral_percentage_999999999_shifted


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lasso,Lasso Regression,1.0376,10.4541,2.6493,-0.8290,0.5399,1.5561,0.6980
en,Elastic Net,1.0376,10.4541,2.6493,-0.8290,0.5399,1.5561,0.1370
dummy,Dummy Regressor,1.0376,10.4541,2.6493,-0.8290,0.5399,1.5561,0.2000
llar,Lasso Least Angle Regression,1.0376,10.4541,2.6493,-0.8290,0.5399,1.5561,0.1350
omp,Orthogonal Matching Pursuit,1.0200,10.2508,2.6199,-1.1153,0.5147,1.3409,0.1370
ada,AdaBoost Regressor,0.9780,10.1882,2.6024,-1.3978,0.4902,1.1600,0.4840
br,Bayesian Ridge,0.9940,10.1780,2.6267,-2.6629,0.5012,1.2286,0.1360
ridge,Ridge Regression,0.9940,10.1779,2.6267,-2.6658,0.5012,1.2286,0.1400
lr,Linear Regression,0.9940,10.1779,2.6267,-2.6660,0.5012,1.2286,0.7790
lar,Least Angle Regression,1.0006,10.1931,2.6343,-3.0986,0.5043,1.2604,0.1370


The best model for cens_electoral_percentage_999999999_shifted is: Lasso(random_state=123)
cens_electoral_percentage_1031_shifted


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
rf,Random Forest Regressor,5.6057,77.9349,8.5935,0.1289,0.7357,0.4499,7.8290
et,Extra Trees Regressor,5.5754,77.4537,8.4739,0.0660,0.7280,0.4439,1.8980
xgboost,Extreme Gradient Boosting,5.8696,80.3865,8.7201,0.0418,0.7661,0.4871,0.2600
lightgbm,Light Gradient Boosting Machine,5.8445,79.4175,8.6907,0.0375,0.7445,0.4764,0.4010
huber,Huber Regressor,6.8264,95.9717,9.5813,-0.0749,0.8164,0.6251,0.1930
omp,Orthogonal Matching Pursuit,6.9934,98.7084,9.7488,-0.1031,0.8717,0.7345,0.1620
gbr,Gradient Boosting Regressor,6.8622,94.5992,9.5346,-0.1085,0.8167,0.6143,2.7060
lar,Least Angle Regression,6.9804,100.4649,9.7885,-0.1381,0.8288,0.6342,0.1620
ridge,Ridge Regression,7.0530,101.5133,9.8654,-0.1446,0.8273,0.6359,0.1570
lr,Linear Regression,7.0530,101.5132,9.8654,-0.1446,0.8273,0.6359,0.1490


The best model for cens_electoral_percentage_1031_shifted is: RandomForestRegressor(n_jobs=-1, random_state=123)
cens_electoral_percentage_86_shifted


...

The best model for cens_electoral_percentage_1000_shifted is: XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cpu', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=-1,
             num_parallel_tree=None, random_state=123, ...)
cens_electoral_percentage_6_shifted


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
lightgbm,Light Gradient Boosting Machine,3.3378,24.4317,4.5765,0.4880,0.4780,0.3478,0.5090
et,Extra Trees Regressor,3.3087,24.2739,4.5628,0.4850,0.4801,0.3438,2.8930
rf,Random Forest Regressor,3.3751,24.9835,4.6577,0.4681,0.4908,0.3549,10.7590
xgboost,Extreme Gradient Boosting,3.4283,25.2413,4.6817,0.4634,0.4923,0.3605,0.3210
gbr,Gradient Boosting Regressor,3.5491,25.8033,4.7539,0.4549,0.4896,0.3750,3.3590
knn,K Neighbors Regressor,3.6589,28.7396,5.0306,0.3548,0.5361,0.3895,0.8460
br,Bayesian Ridge,4.2142,32.0240,5.4242,0.3106,0.5788,0.4567,0.2390
lr,Linear Regression,4.2144,32.0276,5.4245,0.3105,0.5788,0.4568,0.2320
ridge,Ridge Regression,4.2144,32.0272,5.4245,0.3105,0.5788,0.4568,0.2410
lar,Least Angle Regression,4.2383,32.3168,5.4457,0.3053,0.5821,0.4591,0.2360


The best model for cens_electoral_percentage_6_shifted is: LGBMRegressor(n_jobs=-1, random_state=123)
cens_electoral_percentage_693_shifted


,Model,MAE,MSE,RMSE,R2,RMSLE,MAPE,TT (Sec)
xgboost,Extreme Gradient Boosting,0.0170,0.0079,0.0474,-0.0053,0.0303,nan,0.2870
lightgbm,Light Gradient Boosting Machine,0.0164,0.0078,0.0450,-0.0053,0.0287,nan,0.4800
gbr,Gradient Boosting Regressor,0.0390,0.0206,0.0939,-0.3063,0.0565,nan,2.8410
lasso,Lasso Regression,0.7678,2.6870,0.9871,-1.2793,0.4362,nan,0.1920
dummy,Dummy Regressor,0.7678,2.6870,0.9871,-1.2793,0.4362,nan,0.1860
llar,Lasso Least Angle Regression,0.7678,2.6870,0.9871,-1.2793,0.4362,nan,0.1970
en,Elastic Net,0.7678,2.6870,0.9871,-1.2793,0.4362,nan,0.2070
huber,Huber Regressor,0.3227,0.6747,0.4395,-1.6295,0.2087,nan,0.3740
par,Passive Aggressive Regressor,0.6805,2.5268,0.8944,-1.7321,0.3626,nan,0.2140
br,Bayesian Ridge,0.3558,0.7709,0.4840,-1.7416,0.2343,nan,0.2190


The best model for cens_electoral_percentage_693_shifted is: XGBRegressor(base_score=None, booster='gbtree', callbacks=None,
             colsample_bylevel=None, colsample_bynode=None,
             colsample_bytree=None, device='cpu', early_stopping_rounds=None,
             enable_categorical=False, eval_metric=None, feature_types=None,
             gamma=None, grow_policy=None, importance_type=None,
             interaction_constraints=None, learning_rate=None, max_bin=None,
             max_cat_threshold=None, max_cat_to_onehot=None,
             max_delta_step=None, max_depth=None, max_leaves=None,
             min_child_weight=None, missing=nan, monotone_constraints=None,
             multi_strategy=None, n_estimators=None, n_jobs=-1,
             num_parallel_tree=None, random_state=123, ...)
